### 직방의 원룸 매물 정보 수집
- 절차
    - 동이름으로 위도, 경도 수집
    - 위도, 경도 => geohash 변환 (geohash란? 위도, 경도값을 가지고 범위 구함)
    - geohash(영역) => 매물아이디
    - 매물아이디 => 매물정보

이 절차는 강사님이 해보시고 정리하신 것임 (미리 정해진게 아니라는 소리)
    

In [6]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=4d3caa5d701565f6120322f0a75eb1986faba296c2bfafb19a27d2de89deca10
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [7]:
import requests
import pandas as pd
import geohash2

In [8]:
# 1. 동 이름 > 위도, 경도 (직방 페이지에서 동이름 검색 후 => 네트워크에서 search 뜸)

In [9]:
#url 깨진경우 구글=>url encoding 검색해서 decode => 깨진 url이 제대로 나옴

#원래 url(깨짐)
#url = 'https://apis.zigbang.com/v2/search?leaseYn=N&q=%EC%97%AD%EC%82%BC%EB%8F%99&serviceType=%EC%9B%90%EB%A3%B80'
#디코딩한 url
#url = 'https://apis.zigbang.com/v2/search?leaseYn=N&q=역삼동&serviceType=원룸'


In [10]:
addr = '강남구 역삼동' #구까지 써줘서 그 구에 해당하는 동을 검색
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

In [17]:
response = requests.get(url)
response.json() #우리가 필요한 값은 lat, lng
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng #위도, 경도 값 가져옴

(37.495365142822266, 127.03306579589844)

In [3]:
# 2. 위도, 경도 > geohash

In [19]:
#precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

In [4]:
# 3. geohash > 매물 아이디

In [20]:
#개행 방법
#한줄로 너무 긴 코드가 있을 때
#엔터치기 전 맨끝에 \ 를 넣어야 함 
#그런데 \뒤에 공백이 있으면 에러남

url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
url


'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydm6&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

In [22]:
response = requests.get(url)
response

<Response [200]>

In [28]:
items = response.json()["items"]
len(items), items[:2] #list안에 dict가 있는 형태
#1000개가 넘어가면 데이터를 가지고 오지 못하는데,
# 2344개니까 끊어서 가져와야함
# ex) 900개씩

(2344,
 [{'lat': 37.485141261794524, 'lng': 127.01296495741643, 'item_id': 32819635},
  {'lat': 37.485627384450254, 'lng': 127.0136283391101, 'item_id': 32838798}])

In [29]:
ids = [item["item_id"] for item in items ]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [31]:
url = 'https://apis.zigbang.com/v2/items/list'

#params에 어떻게 넣어야 하는지는 payload에 있음
#post 방식일 때는 params를 써줘야함.
#get 방식일 때는 params를 안써도 됨
params = {
    "domain": "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids[:900]
}

#이 params에는 한글이 없기 때문에 json.dumps()로 인코딩할 필요가 없다.
response = requests.post(url, params)
response

<Response [200]>

In [33]:
response.text[:300] #items에 리스트 형태로 데이터가 들어있음

'{"items":[{"section_type":null,"item_id":32819635,"images_thumbnail":"https://ic.zigbang.com/ic/items/32819635/1.jpg","sales_type":"전세","sales_title":"전세","deposit":16500,"rent":0,"size_m2":39.89,"공급면적":{"m2":39.89,"p":"12.1"},"전용면적":{"m2":20.82,"p":"6.3"},"계약면적":null,"room_type_title":null,"floor":'

In [45]:
# max row, max column 설정
pd.options.display.max_columns, pd.options.display.max_rows
#pd.options.display.max_columns=50

(50, 60)

In [37]:
items = response.json()["items"]
#items[:2]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9
899,32656329,전세,24000,0,59.50,서울시 강남구 역삼동,10


In [46]:
# 함수 만들기
def oneroom(addr) :
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    response.json() #우리가 필요한 값은 lat, lng
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items ]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
    "domain": "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]
    

In [49]:
addr = "마포구 합정동"
df=oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
743,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [55]:
# '마포구 합정동'이 들어간 데이터만 True/False
# True인 것만 가져오기
# reset_index는 그냥 가져오면 맨 앞에 인덱스 값이 기존 인덱스이기때문에 
# 인덱스 넘버를 바꿈
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [59]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7C6C-FC35

 C:\Users\User\AIVLESCHOOLPythonClass\python_crawling\Code 디렉터리

2022-08-04  오후 01:33    <DIR>          .
2022-08-04  오후 01:33    <DIR>          ..
2022-08-04  오전 11:08    <DIR>          .ipynb_checkpoints
2022-08-04  오전 11:07            62,765 01_naver_api_2.ipynb
2022-08-03  오후 03:40            93,268 01_requests_json.ipynb
2022-08-03  오후 05:09            23,447 02_requests_api.ipynb
2022-08-03  오후 05:10             9,016 02_requests_daum_exchange.ipynb
2022-08-04  오후 01:33            25,861 02_zigbang.ipynb
2022-08-03  오후 04:46            10,060 covid.xlsx
2022-08-03  오후 04:51             6,132 covid_en.xlsx
2022-08-04  오후 01:33             1,249 zigbang.py
               8개 파일             231,798 바이트
               3개 디렉터리  108,350,857,216 바이트 남음


In [56]:
#모듈 파일 만들기 : .py

In [58]:
%%writefile zigbang.py 
#magic command에 의해서 아래의 코드가 python코드로 저장이 된다.
#함수에서 사용되는 패키지도 같이 추가해야 한다.

import requests
import pandas as pd
import geohash2

def oneroom(addr) :
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    response.json() #우리가 필요한 값은 lat, lng
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items ]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
    "domain": "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]
    

Writing zigbang.py


In [60]:
import zigbang as zb

df = zb.oneroom('망원동')
df

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32864713,전세,9000,0,21.49,서울시 영등포구 당산동4가,13
1,32694522,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
2,32822043,전세,9000,0,16.53,서울시 영등포구 당산동4가,13
3,32827028,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
4,32856780,전세,7000,0,19.83,서울시 영등포구 당산동4가,13
...,...,...,...,...,...,...,...
741,32831032,월세,200,65,18.18,서울시 마포구 성산동,7
742,32775788,전세,16500,0,19.83,서울시 마포구 성산동,1
743,32589981,전세,43000,0,49.00,서울시 마포구 중동,7
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5


In [63]:
%whos
# 사용하고 있는 변수 목록을 보여줌

Interactive namespace is empty.


In [62]:
%reset
#변수를 리셋시킴

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
